# WGS-Mutant_strain_ex1-Hawaiian_tutorial
* [Amhed Missael Vargas Velazquez](https://www.researchgate.net/profile/Amhed-Vargas-Velazquez)
* Post-doctoral fellow, [SGB lab](https://syngenbio.kaust.edu.sa/), [KAUST](https://www.kaust.edu.sa/en)

## Description
This jupyter notebook will guide you through the analysis required to identify the causal mutation for a *roller* phenotype in two [mapping by sequencing](https://www.genetics.org/content/204/2/451) experiments ([Smith *et al.* 2016](https://www.g3journal.org/content/6/5/1297)). In these experiments, the strategy used was *Hawaiian* mapping, therefore, a `.vcf` file containing the variants of CB4856 is required to run this pipeline.  

## Software requirements
The core instance running this script is Python. However, most of the analysis are performed by other programs (handled by `system calls !`) which have to be installed or, for portability convenience, be present in a folder (check `WGS-Software_configuration` notebook for further details). Note that software in **bold** are the only ones we distribute, the rest must be installed in the computer running this notebook.

* [samtools](http://www.htslib.org/)
* [bwa](https://github.com/lh3/bwa)
* [**GATK**](https://gatk.broadinstitute.org/hc/en-us) [3.7+](https://console.cloud.google.com/storage/browser/gatk-software/package-archive/gatk/); note that current version of GATK (4.0+) uses different nomenclature and have different tools which makes it not compatible with this pipeline. We describe in detail why we stick to this software in the notebook `WGS-Software_configuration`.
* [**picard**](https://broadinstitute.github.io/picard/)
* [**snpEff**](https://pcingola.github.io/SnpEff/)
* [fastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/)
* [SRAToolkit](https://github.com/ncbi/sra-tools)
* wget
* zcat
* [R](https://cran.r-project.org/)
* perl

### Python libraries
* os
* IPython.display
* numpy
* pandas
* matplotlib

### R libraries
* ggplot2

## Getting started
Run the cells below to load essential python libraries, access a working directory, and to verify you have the required software

### Load python libraries
Run the cell below to load essential libraries for the pipeline to work:

In [1]:
## Load libraries
#os to move within directories
import os
#IPython.display for markdown
from IPython.display import display, Markdown
#matplotlib, pandas, and numpy for plotting and data analysis respectively
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Select a folder that can be accesed by the pipeline  
Before starting any analysis, make sure to select a folder where the analysis will be performed (*unless* stated otherwise, the analysis will be performed on the **same folder as this notebook**):

In [2]:
##Set working directory
#Same location as script
path = os.getcwd()
#or somewhere else, e.g.:
#path = '/home/jupyter-USER/Workstation/USER/parental'

##Move to path
os.chdir(path)

##Show current directory to user
display(Markdown('<div class=\"alert alert-block alert-info\">Directory for analysis:<br><b>' + os.getcwd() + '</b></div>'))

<div class="alert alert-block alert-info">Directory for analysis:<br><b>/home/velazqam/Projects/jupyter_notebooks/dev</b></div>

### Make sure to have the required "stand alone" software
Most of the programs used within this pipeline have "stand-alone" versions that allow users to run their analysis on any computer they want. However, first you have to make sure to have those programs. Particularly, make sure to have a directory containing the following ones: 

- GATK (GenomeAnalysisTK.jar)
- Picard (picard.jar)
- SnpEff (folder with both snpEff.jar and SnpSift.jar, and another folder with its database; more below)

For your convenience, there is a folder already containing these programs. Just make sure to set properly the path to them, e.g.:

In [3]:
##Path to software folder
softpath = '/home/WGS_pipeline/Software'

##GATK v3.8.1.0
GATKpath = (softpath + '/gatk-3.8.1.0')

##Picard v2.23.6 
PiKpath = (softpath + '/picard2.23.6')

##SnpEff v.0
Snpath = (softpath + '/SnpEff-5.0/snpEff')

##Alternative paths
#GATKpath = ''
#PiKpath = ''
#Snpath = ''

##Check if jar files are there
##Notify user if GATK .jar is present or not
if os.path.isfile(GATKpath + '/GenomeAnalysisTK.jar'):
    display(Markdown('<div class=\"alert alert-block alert-success\"><b>\b GATK</b></div>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>GATK not found in: ' + GATKpath +'</div>'))

##Notify user if Picard .jar is present or not
if os.path.isfile(PiKpath + '/picard.jar'):
    display(Markdown('<div class=\"alert alert-block alert-success\"><b>\b Picard</b></div>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Picard not found in: ' + PiKpath +'</div>'))

##Notify user if SnpEff .jar is present or not
if os.path.isfile(Snpath + '/snpEff.jar'):
    display(Markdown('<div class=\"alert alert-block alert-success\"><b>\b SnpEff</b></div>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>SnpEff not found in: ' + GATKpath +'</div>'))

<div class="alert alert-block alert-success"><b> GATK</b></div>

<div class="alert alert-block alert-success"><b> Picard</b></div>

<div class="alert alert-block alert-success"><b> SnpEff</b></div>

### Make sure to have a reference genome
In order to run this pipeline, a *C. elegans* reference genome is needed (preferentially in fasta format). The cells below allows you to prepare your reference file or to download the *C. elegans* ce11/WS235 version from Ensemble.

First, download or specify the location of your reference genome:

In [4]:
##Specify the location of the genome of Reference
#e.g.
RefFile=('/home/jupyter-newuser/Workstation/Amhed/Caenorhabditis_elegans.fa')

###OR

##To download ce11 uncomment (remove the # sign) from the lines below

#!mkdir -p {path}/data

#os.chdir(path+'/data')

#!wget -q ftp://ftp.ensembl.org/pub/release-99/fasta/caenorhabditis_elegans/dna/Caenorhabditis_elegans.WBcel235.dna_sm.toplevel.fa.gz

#!zcat Caenorhabditis_elegans.WBcel235.dna_sm.toplevel.fa.gz > Caenorhabditis_elegans.WBcel235.99.softmasked.fa

#RefFile=(path+'/data/'+'Caenorhabditis_elegans.WBcel235.99.softmasked.fa')

##Notify user if file present or not
if os.path.isfile(RefFile):
    display(Markdown('<div class=\"alert alert-block alert-success\"><b>Reference Genome :</b>\n'+RefFile+'</div>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Reference file: '+RefFile+' does not exists</div>'))

<div class="alert alert-block alert-success"><b>Reference Genome :</b>
/home/jupyter-newuser/Workstation/Amhed/Caenorhabditis_elegans.fa</div>

Then it need to be indexed by samtools:

In [5]:
##Check first if file is indexed already
if os.path.isfile(RefFile+'.fai'):
    display(Markdown('<b>\b</b>'))
else:
    !samtools faidx {RefFile}
    ##Notify user if this step was sucessfull
    if os.path.isfile(RefFile+'.fai'):
        display(Markdown('<b>\b</b>'))
    else:
            display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Index file not produced for '+RefFile+'</div>'))

<b></b>

And bwa

In [6]:
##Check first if file is indexed already
if os.path.isfile(RefFile+'.bwt'):
    display(Markdown('<b>\b</b>'))
else:
    !bwa index {RefFile} > /dev/null 2>&1
    ##Notify user if this step was sucessfull
    if os.path.isfile(RefFile+'.bwt'):
        display(Markdown('<b>\b</b>'))
    else:
        display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Index file not produced for '+RefFile+'</div>'))

<b></b>

And by picard tools

In [7]:
RefFilebase=''.join(RefFile.split('.')[:-1])

##Check first if file is indexed already
if os.path.isfile(RefFilebase +'.dict'):
    display(Markdown('<b>\b</b>'))
else:
    !java -jar {softpath}/picard2.23.6/picard.jar CreateSequenceDictionary R={RefFile} O={RefFilebase +'.dict'} > /dev/null 2>&1
    ##Notify user if this step was sucessfull
    if os.path.isfile(RefFilebase +'.dict'):
        display(Markdown('<b>\b</b>'))
    else:
        display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Index file not produced for '+RefFile+'</div>'))

<b></b>

Now your reference genome is ready for the analysis in this pipeline (**as well as many others**).

In [8]:
ReferenceGenome = RefFile

## Brief introduction
Whole Genome Sequencing (WGS) has revolutionized how we identify genes that act in the development of a phenotype. For a short historical review check this [timeline of *C. elegans* research](https://www.hobertlab.org/wp-content/uploads/2013/03/Ankeny_2001.pdf) and for a better understanding read the Wormbook chapter on [NGS methods to map mutations in *C. elegans*](https://www.genetics.org/content/204/2/451).

This jupyter notebook will guide you through the computation used to identify the causing mutation of a *roller* phenotype using two mapping by sequencing approaches. In particular, we will map genetic mutations that have been introgressed to the *Hawaiian* (CB4856) genomic background. As a first exercise, we will replicate the **Fig. 2** of [Smith et al.](https://www.g3journal.org/content/6/5/1297) which describes two ways to map a mutation located in *rol-6* (see image below). In the `WGS-Mutant_strain_ex2-Hawaiian_tutorial` notebook, multiple strains will be analized at once, so refer to it for an advance use of this pipeline. Please note that a *.vcf* (variant call format) file containing the genomic variants of **CB4856** is needed to run this notebook; in case you're interested on how to produce this file, please refer to the `WGS-Parental_strain-Hawaiian_tutorial` jupyter notebook.

![Fig.2 https://doi.org/10.1534/g3.116.028316](https://www.g3journal.org/content/ggg/6/5/1297/F2.large.jpg)

#### Image caption from Fig2 of [Smith *et al.* 2016](https://www.g3journal.org/content/6/5/1297):
> Polymorphism mapping of dominant mutations. **A. Mapping strategies.** The strain bearing the dominant mutation (asterisk, in red) is crossed to the mapping strain (haw, in green). For reverse mapping, wild-type F2 progeny are pooled for sequencing. For F3 screening, progeny from individual mutant F2s are screened to identify homozygous mutants, which are pooled for sequencing. **B. Reverse mapping data for *rol-6*(*su1006*).** Plot is shown for chromosome II, which contains *rol-6* at position 8.7 Mb. **C. Mapping data for F3 screen.** Chromosome II, with *rol-6* indicated.

### Obtaining sequencing reads from NCBI Short Read Archive (SRA)

In order to fully replicate the figure above, we need the sequencing data of the bulk segregants in the "Reverse mapping" and the "F3 screen". Fortunately, these files were submitted to the SRA with the accesion numbers [SRR3084251](https://www.ncbi.nlm.nih.gov/sra/?term=SRR3084251) and [SRR3084348](https://www.ncbi.nlm.nih.gov/sra/?term=SRR3084348) respectively.

We will use `wget` to download the SRA files and the `SRAToolkit` to convert them back to fastq files. Please note that these libraries are single-end (only a single fastq file will be obtained per SRA file)

In [9]:
##rol-6 reverse mapping
#SRR3084251
#!wget -q https://sra-pub-run-odp.s3.amazonaws.com/sra/SRR3084251/SRR3084251

##rol-6 F3 screen
#SRR3084348
#!wget -q https://sra-pub-run-odp.s3.amazonaws.com/sra/SRR3084348/SRR3084348
    
##Dump files into fastq files
#!fastq-dump SRR3084251
#!fastq-dump SRR3084348

Alternatively, you can copy the sequencing reads from our internal folder:

In [9]:
##Copy from internal folder
!cp /home/WGS_pipeline/data/SRR3084251.fastq .
!cp /home/WGS_pipeline/data/SRR3084348.fastq .

In [10]:
##Verify if files were properly obtained
#SRR2003569_1.fastq
if os.path.isfile('SRR3084251.fastq'):
    display(Markdown('<b>SRR3084251.fastq \b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>The file SRR3084251.fastq is not present in current path: '+os.getcwd()+'</div>'))

#SRR2003569_2.fastq
if os.path.isfile('SRR3084348.fastq'):
    display(Markdown('<b>SRR3084348.fastq \b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>The file SRR3084348.fastq is not present in current path: '+os.getcwd()+'</div>'))

<b>SRR3084251.fastq </b>

<b>SRR3084348.fastq </b>

### Hawaiian (CB4856) genomic variants

In order to fully replicate the figure above, we need first to identify the genomic variants seen in the bulk segregants for the "Reverse mapping" and the "F3" screens. Subsequently, we need to subtract the genomic variants that belongs to the Hawaiian strain. These variants (formated into a `.vcf` file) can be obtained by following the `WGS-Parental_strain-Hawaiian_tutorial` notebook. Alternatively, you can use `.vcf` files from other sources (see below).

In [ ]:
## Path to .vcf file containing CB4856 variants
#ParentVcf = "/home/jupyter-user/analysis/data/CB4856.UG.vcf"

Alternatevily, you can copy a vcf file containing Hawaiian genomic variants from sequencing runs produced for wild isolates at [Andersen lab collection](https://trace.ncbi.nlm.nih.gov/Traces/sra/?run=SRR2003569), or from [CeNDR](https://elegansvariation.org/data/release/latest), or from wormbase. Plea but (liftover) to a more recent version of C. elegans gneome.

## Data analysis workflow
The common pipeline consist on:
* Quality assesment of sequencing reads via FastQC
* Mapping of reads with bwa
* Filtering and processing of alignment file with samtools
* Coverage analysis with Samtools
* Realigment with GATK
* Variant calling with GATK HC and UG
* Mutational analysis with SnpEff

But before that, lets create a directory where the analysis will be executed:

In [ ]:
##Move to that directory
os.chdir(path)
##Create directory to place data
!mkdir -p {path}/analysis
##Move to that directory
os.chdir( path + '/analysis' )

Lets now define parameters for pipeline:

In [ ]:
##Inputs
SampleName = 'CFJ125'
FastqF1 = ('/home/jupyter-newuser/Workstation/CFJ125/CFJ125_R1.fastq') 
FastqF2 = ('/home/jupyter-newuser/Workstation/CFJ125/CFJ125_R2.fastq')
SnpEffGen = 'WBcel235.99'

#Minimal quality for bam
minMaqQforBam = 1
#Minimum quality for vcf filtering
minMaqQforVcf = 10
#Minimum basequality for variant calling
minBaseQforVcf = 10
#
minVarCall = 10
diffStep = 10
minDel = 2
maxSample= 2

#Number of threats
Ncpu=1
#Ram for java
ramG=20

### Quality assesment of Fastq files
A simple way to verify the quality of your sequencing runs is via the FastQC program. The following cells will make a directory to run the analysis and output its results.

In [ ]:
##Create directory to place data
!mkdir -p FastQC
##Move to that directory
os.chdir(path + '/analysis/FastQC')

In [ ]:
##Perform fastQC analysis
!fastqc -t {Ncpu} {FastqF1} {FastqF2} -o . > /dev/null 2>&1

##Notify user if this step was sucessfull
tempname1 = FastqF1.split("/")[-1]
tempname1 = ''.join(tempname1.split(".")[-len(tempname1.split("."))])
if os.path.isfile(tempname1 + '_fastqc.html'):
    #display(Markdown('<div class=\"alert alert-block alert-success\">'))
    !unzip -o -qq \*.zip
    display(Markdown('<b>Metrics:</b>'))
    tempname1 = FastqF1.split("/")[-1]
    tempname1 = ''.join(tempname1.split(".")[-len(tempname1.split("."))])
    print(tempname1)
    !cat {tempname1}_fastqc/summary.txt
    tempname2 = FastqF2.split("/")[-1]
    tempname2 = ''.join(tempname2.split(".")[-len(tempname2.split("."))])
    print(tempname2)
    !cat {tempname2}_fastqc/summary.txt
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>FASTQC report not produced in current directory: '+os.getcwd()+'</div>'))

#### HTML reports
Run the cell below to link the FastQC results to this notebook.

In [ ]:
display(Markdown('Full reports (open a new tab to see them): \n' + '* [' + tempname1 + '](./analysis/FastQC/' + tempname1 +'_fastqc.html)\n' + '* [' + tempname2 + '](./analysis/FastQC/' + tempname2 +'_fastqc.html)\n'))

### Mapping reads to reference genome using bwa

After assesed the quality of the Fastq reads, we will map them to the reference genome using bwa. For that running the cells belows will produce a directory where the analysis will be performed.

In [ ]:
##Move to analysis
os.chdir(path + '/analysis/')
##Create directory to place data
!mkdir -p Alignments
##Move to that directory
os.chdir(path + '/analysis/Alignments')

In [ ]:
!bwa mem -t {Ncpu} -M {ReferenceGenome} {FastqF1} {FastqF2} -o {SampleName}.sam > /dev/null 2>&1

##Notify user if previous step was sucessfull
if os.path.isfile(SampleName+'.sam'):
    #display(Markdown('<div class=\"alert alert-block alert-success\">\b</div>'))
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Sam file not present in current path: '+os.getcwd()+'</div>'))

Now lets transform to bam

In [ ]:
!samtools view -@ {Ncpu} -S -bh {SampleName}.sam | samtools sort -@ {Ncpu} - > {SampleName}.bam

if os.path.isfile(SampleName+'.bam'):
    #display(Markdown('<div class=\"alert alert-block alert-success\">\b</div>'))
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Sam file not present in current path: '+os.getcwd()+'</div>'))

Now lets index

In [ ]:
!samtools index {SampleName}.bam

if os.path.isfile(SampleName+'.bam.bai'):
    #display(Markdown('<div class=\"alert alert-block alert-success\">\b</div>'))
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Index file not present in current path: '+os.getcwd()+'</div>'))

Filter for mapping quality

In [ ]:
!samtools view -@ {Ncpu} -q {minMaqQforBam} -bh {SampleName}.bam > {SampleName}.SF.bam

if os.path.isfile(SampleName+'.SF.bam'):
    #display(Markdown('<div class=\"alert alert-block alert-success\">\b</div>'))
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Filtered file not present in current path: '+os.getcwd()+'</div>'))

Then add and replace groups via picard

In [ ]:
!java -Xmx{ramG}g -jar {softpath}/picard2.23.6/picard.jar AddOrReplaceReadGroups I={SampleName}.SF.bam O={SampleName}.RG.bam RGID={SampleName} RGLB=LB RGPL=illumina RGPU=PU RGSM={SampleName} > /dev/null 2>&1

if os.path.isfile(SampleName+'.RG.bam'):
    #display(Markdown('<div class=\"alert alert-block alert-success\">\b</div>'))
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Read groups file not present in current path: '+os.getcwd()+'</div>'))

Then we mark duplicates

In [ ]:
!java -Xmx{ramG}g -jar {softpath}/picard2.23.6/picard.jar MarkDuplicates I={SampleName}.RG.bam O={SampleName}.Dup.bam M={SampleName}.dedupMetrics REMOVE_DUPLICATES=true > /dev/null 2>&1

if os.path.isfile(SampleName+'.Dup.bam'):
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Aligment withouth duplicates file not present in current path: '+os.getcwd()+'</div>'))

Index bam

In [ ]:
!samtools index {SampleName}.Dup.bam

if os.path.isfile(SampleName+'.Dup.bam.bai'):
    #display(Markdown('<div class=\"alert alert-block alert-success\">\b</div>'))
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Index file not present in current path: '+os.getcwd()+'</div>'))

Feel free to download the files before realigment by following the links:

In [ ]:
display(Markdown('Download bam files (right-click and \"save as\"): \n' + '* [' + SampleName + '.Dup.bam](./analysis/Alignments/' + SampleName +'.Dup.bam)\n' + '* [' + SampleName + '.Dup.bam.bai](./analysis/Alignments/' + SampleName +'.Dup.bam.bai)\n'))

### Coverage analysis with samtools
Finally let's see basic stats of the reads mapped in the last bam file

In [ ]:
!samtools coverage {SampleName}.Dup.bam

### Local realigment of reads around indels via GATK
Let's now proceed to re-align the reads to do better calling around indels. For that, we start by identifying the location of possible indes via GATK

In [ ]:
!java -Xmx{ramG}g -jar {softpath}/gatk-3.8.1.0/GenomeAnalysisTK.jar -T RealignerTargetCreator -nt {Ncpu} -R {ReferenceGenome} -I {SampleName}.Dup.bam -o {SampleName}.Indel.intervals > /dev/null 2>&1

if os.path.isfile(SampleName+'.Indel.intervals'):
    #display(Markdown('<div class=\"alert alert-block alert-success\">\b</div>'))
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>File with realigned indels not present in current path: '+os.getcwd()+'</div>'))

And then we proceed to perform realigment around the spotted regions

In [ ]:
!java -Xmx{ramG}g -jar {softpath}/gatk-3.8.1.0/GenomeAnalysisTK.jar -T IndelRealigner -R {ReferenceGenome} -I {SampleName}.Dup.bam -targetIntervals {SampleName}.Indel.intervals -o {SampleName}.realigned.bam > /dev/null 2>&1

if os.path.isfile(SampleName+'.realigned.bam'):
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Realigned bam not present in current path: '+os.getcwd()+'</div>'))

Check if files present

In [ ]:
if os.path.isfile(SampleName+'.realigned.bam'):
    display(Markdown('<div class=\"alert alert-block alert-success\"><b>\b Mapping step complete</b></div>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Realigned bam not present in current path: '+os.getcwd()+'<br>Either the path is wrong or cells above have to be run again</div>'))

Now is time for variant calling, though feel free to check the mapping file using tablet/igv. You can download the bam and the index file from:

In [ ]:
display(Markdown('Download bam files (right-click and \"save as\"): \n' + '* [' + SampleName + '.realigned.bam](./analysis/Alignments/' + SampleName +'.realigned.bam)\n' + '* [' + SampleName + '.realigned.bam.bai](./analysis/Alignments/' + SampleName +'.realigned.bam.bai)\n'))

### Variant calling
Now that our reads are properly aligned, lets do variant calling. To start, let's make a new directory where the analysis will be run and the results reported.

In [ ]:
##Move to analysis
os.chdir(path + '/analysis/')
##Create directory to place data
!mkdir -p VariantCalling
##Move to that directory
os.chdir(path + '/analysis/VariantCalling')

This pipeline uses two different GATK variant callers, those being Unified Genotyper (UG) and Haplotype Caller (HC). Both should produce good calls though some pipelines preffer the use of HC given its confidence metrics.

#### Unified genotyper
The code below produces a vcf via GATK's unified genotyper tool

In [ ]:
!java -Xmx{ramG}g -jar {softpath}/gatk-3.8.1.0/GenomeAnalysisTK.jar -T UnifiedGenotyper -R {ReferenceGenome} -nt {Ncpu} -l INFO -glm BOTH -I {path}/analysis/Alignments/{SampleName}.realigned.bam -o {SampleName}.UG.vcf -mbq {minBaseQforVcf} -stand_call_conf {minVarCall} > /dev/null 2>&1

if os.path.isfile(SampleName+'.UG.vcf'):
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Realigned bam not present in current path: '+os.getcwd()+'</div>'))

#### Haplotype caller
Before calling variants vie GATK haplotype caller, we need to produce confidence values for each site and store them into a g.vcf file

In [ ]:
!java -Xmx{ramG}g -jar {softpath}/gatk-3.8.1.0/GenomeAnalysisTK.jar -T HaplotypeCaller --emitRefConfidence GVCF -R {ReferenceGenome} -l INFO -I {path}/analysis/Alignments/{SampleName}.realigned.bam -o {SampleName}.g.vcf -mbq {minBaseQforVcf} -mmq {minMaqQforVcf} -stand_call_conf {minVarCall} > /dev/null 2>&1

if os.path.isfile(SampleName+'.g.vcf'):
    display(Markdown('<b>\b</b>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Realigned bam not present in current path: '+os.getcwd()+'</div>'))

After that, we can proceed to call the vcf from the g.vcf

In [ ]:
!java -Xmx{ramG}g -jar {softpath}/gatk-3.8.1.0/GenomeAnalysisTK.jar -T GenotypeGVCFs -R {ReferenceGenome} -nt {Ncpu} -l INFO -V {SampleName}.g.vcf -dt none -o {SampleName}.HC.vcf  > /dev/null 2>&1
   
    ## Check if HC vcf is done
if os.path.isfile(SampleName+'.HC.vcf'):
    display(Markdown('<div class=\"alert alert-block alert-success\"><b>\b Haplotype Caller complete</b></div>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Realigned bam not present in current path: '+os.getcwd()+'<br>Either the path is wrong or cells above have to be run again</div>'))
    
## Check if UG vcf is done
if os.path.isfile(SampleName+'.UG.vcf'):
    display(Markdown('<div class=\"alert alert-block alert-success\"><b>\b Unified Genotypifier complete</b></div>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Realigned bam not present in current path: '+os.getcwd()+'<br>Either the path is wrong or cells above have to be run again</div>'))


Now, for each vcf let's plot the distribution of the analized variants with python. First the vcf produced with haplotype caller

In [ ]:
df = pd.read_csv((SampleName+'.HC.vcf'),delimiter='\t',comment='#', names=["chr","pos","id","ref","alt","qual","filter","info","format",SampleName])

strarray= df[SampleName]

RefD=[]
AltD=[]
Freq=[]
for i in range(len(strarray)):
    deps=(strarray[i].split(':'))[1].split(',')
    RefD.append(int(deps[0]))
    AltD.append(int(deps[1]))
    tmpfre=int(deps[0]) + int(deps[1])
    if tmpfre > 0 :
        Freq.append(int(deps[1])/(int(deps[0]) + int(deps[1])))
    else :
        Freq.append(0)

df['RefD']=RefD
df['AltD']=AltD
df['Freq']=Freq

##Plot as separated interactive plot
%matplotlib inline
##Or non interactive
#%matplotlib notebook

Window = 50000
AFreq = .8
display(Markdown('<b>'+SampleName+' Haplotype Caller; Number of variants above an allele frequency >'+str(AFreq)+' in '+str(Window)+'bp non-overlapping windows</b>'))

for chrname in df["chr"].unique() :
    chr1=df.loc[df["chr"]==chrname]
    chr1 = chr1.loc[chr1["Freq"]>AFreq]
    chr1H = np.array((chr1.groupby(chr1["pos"] // Window).count())["chr"])
    plt.figure()
    plt.stem(np.array(range(len(chr1H)))*Window,chr1H)
    plt.xlabel("Genomic Location, Window Size = " + str(Window))
    plt.ylabel("No. of variants with AF > " + str(AFreq))
    plt.title(chrname)
plt.show() 

#Plot all together
#%matplotlib notebook
#%matplotlib inline
#Window = 50000
#AFreq = .8
#con = 0
#figure, axis = plt.subplots((len(df["chr"].unique()) + 3)//4, 4)
#for chrname in df["chr"].unique() :
#    chr1=df.loc[df["chr"]==chrname]
#    chr1 = chr1.loc[chr1["Freq"]>AFreq]
#    chr1H = np.array((chr1.groupby(chr1["pos"] // Window).count())["chr"])
#    axis[con//4, con%4].stem(np.array(range(len(chr1H)))*Window,chr1H)
#    axis[con//4, con%4].set_title(chrname)
#    con = con + 1
#plt.show()

And then unified genotyper

In [ ]:
df = pd.read_csv((SampleName+'.UG.vcf'),delimiter='\t',comment='#', names=["chr","pos","id","ref","alt","qual","filter","info","format",SampleName])

strarray= df[SampleName]

RefD=[]
AltD=[]
Freq=[]
for i in range(len(strarray)):
    deps=(strarray[i].split(':'))[1].split(',')
    RefD.append(int(deps[0]))
    AltD.append(int(deps[1]))
    tmpfre=int(deps[0]) + int(deps[1])
    if tmpfre > 0 :
        Freq.append(int(deps[1])/(int(deps[0]) + int(deps[1])))
    else :
        Freq.append(0)

df['RefD']=RefD
df['AltD']=AltD
df['Freq']=Freq

##Plot as separated interactive plot
%matplotlib inline
##Or non interactive
#%matplotlib notebook

Window = 50000
AFreq = .8
display(Markdown('<b>'+SampleName+' Unified Genotyper; Number of variants above an allele frequency >'+str(AFreq)+' in '+str(Window)+'bp non-overlapping windows</b>'))

for chrname in df["chr"].unique() :
    chr1=df.loc[df["chr"]==chrname]
    chr1 = chr1.loc[chr1["Freq"]>AFreq]
    chr1H = np.array((chr1.groupby(chr1["pos"] // Window).count())["chr"])
    plt.figure()
    plt.stem(np.array(range(len(chr1H)))*Window,chr1H)
    plt.xlabel("Genomic Location, Window Size = " + str(Window))
    plt.ylabel("No. of variants with AF > " + str(AFreq))
    plt.title(chrname)
plt.show() 

#Plot all together
#%matplotlib notebook
#%matplotlib inline
#Window = 50000
#AFreq = .8
#con = 0
#figure, axis = plt.subplots((len(df["chr"].unique()) + 3)//4, 4)
#for chrname in df["chr"].unique() :
#    chr1=df.loc[df["chr"]==chrname]
#    chr1 = chr1.loc[chr1["Freq"]>AFreq]
#    chr1H = np.array((chr1.groupby(chr1["pos"] // Window).count())["chr"])
#    axis[con//4, con%4].stem(np.array(range(len(chr1H)))*Window,chr1H)
#    axis[con//4, con%4].set_title(chrname)
#    con = con + 1
#plt.show()

Finally, let's proceed to annotate each variant

### Variant annotation with SnpEff
The last step is to identify if any mutation observed has an effect over any coding sequence. For that, we use SNPeff suite. Lets start by creating a new folder and going there:

In [ ]:
##Move to analysis
os.chdir(path + '/analysis/')
##Create directory to place data
!mkdir -p VariantPrediction
##Move to that directory
os.chdir(path + '/analysis/VariantPrediction')

Then, let's run SnpEff on both vcfs produced by GATK algorithms.

In [ ]:
###Produce Annotations
!java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/snpEff.jar {SnpEffGen} -stats {SampleName}.HC.html {path}/analysis/VariantCalling/{SampleName}.HC.vcf > {SampleName}.HC.ann.vcf
!java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/snpEff.jar {SnpEffGen} -stats {SampleName}.UG.html {path}/analysis/VariantCalling/{SampleName}.UG.vcf > {SampleName}.UG.ann.vcf 

if os.path.isfile(SampleName+'.HC.ann.vcf'):
    if os.path.isfile(SampleName+'.UG.ann.vcf'):
        display(Markdown('<b>\b</b>'))
    else:
        display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Annotated UG vcf not present in current path: '+os.getcwd()+'</div>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Annotated HC vcf not present in current path: '+os.getcwd()+'</div>'))

In [ ]:
!cat {SampleName}.HC.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - -s "," -e "." CHROM POS REF ALT QUAL GEN[{SampleName}].GT GEN[{SampleName}].DP GEN[*].AD "ANN[*].GENEID" "ANN[*].ALLELE" "ANN[*].EFFECT" "ANN[*].IMPACT" "ANN[*].GENE" "ANN[*].FEATURE" "ANN[*].FEATUREID" "ANN[*].BIOTYPE" "ANN[*].RANK" "ANN[*].HGVS_C" "ANN[*].HGVS_P" "ANN[*].CDNA_POS" "ANN[*].CDNA_LEN" "ANN[*].CDS_POS" "ANN[*].CDS_LEN" "ANN[*].AA_POS" "ANN[*].AA_LEN" "ANN[*].DISTANCE" "LOF[*].GENE" "LOF[*].GENEID" "NMD[*].GENE" "NMD[*].GENEID" > {SampleName}.HC.ann.txt
!cat {SampleName}.UG.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - -s "," -e "." CHROM POS REF ALT QUAL GEN[{SampleName}].GT GEN[{SampleName}].DP GEN[*].AD "ANN[*].GENEID" "ANN[*].ALLELE" "ANN[*].EFFECT" "ANN[*].IMPACT" "ANN[*].GENE" "ANN[*].FEATURE" "ANN[*].FEATUREID" "ANN[*].BIOTYPE" "ANN[*].RANK" "ANN[*].HGVS_C" "ANN[*].HGVS_P" "ANN[*].CDNA_POS" "ANN[*].CDNA_LEN" "ANN[*].CDS_POS" "ANN[*].CDS_LEN" "ANN[*].AA_POS" "ANN[*].AA_LEN" "ANN[*].DISTANCE" "LOF[*].GENE" "LOF[*].GENEID" "NMD[*].GENE" "NMD[*].GENEID" > {SampleName}.UG.ann.txt

if os.path.isfile(SampleName+'.HC.ann.txt'):
    if os.path.isfile(SampleName+'.UG.ann.txt'):
        display(Markdown('<b>\b</b>'))
    else:
        display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Annotated UG vcf not present in current path: '+os.getcwd()+'</div>'))
else:
    display(Markdown('<div class=\"alert alert-block alert-danger\"><b>Error:</b><br>Annotated HC vcf not present in current path: '+os.getcwd()+'</div>'))

In [ ]:
## other approach
!echo "#High impact mutations" > {SampleName}.HC.ann.sort.txt
!cat {SampleName}.HC.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'HIGH'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.HC.ann.sort.txt
!echo "#Moderate impact mutations" >> {SampleName}.HC.ann.sort.txt
!cat {SampleName}.HC.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'MODERATE'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.HC.ann.sort.txt
!echo "#Low impact mutations" >> {SampleName}.HC.ann.sort.txt
!cat {SampleName}.HC.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'LOW'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.HC.ann.sort.txt

!echo "#High impact mutations" > {SampleName}.UG.ann.sort.txt
!cat {SampleName}.UG.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'HIGH'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.UG.ann.sort.txt
!echo "#Moderate impact mutations" >> {SampleName}.UG.ann.sort.txt
!cat {SampleName}.UG.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'MODERATE'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.UG.ann.sort.txt
!echo "#Low impact mutations" >> {SampleName}.UG.ann.sort.txt
!cat {SampleName}.UG.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'LOW'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.UG.ann.sort.txt

!echo "First 10 entries in Unified Genotyper variants"
!head {SampleName}.UG.ann.sort.txt
!echo "First 10 entries in Haplotype Caller"
!head {SampleName}.HC.ann.sort.txt

In [ ]:
display(Markdown('Full reports (open a new tab to see them): \n' + '* [' + SampleName + ' Unified Genotyper results](./analysis/VariantPrediction/' + SampleName +'.UG.html)\n' + '* [' + SampleName + ' Haplotype Caller results](./analysis/VariantPrediction/' + SampleName +'.HC.html)\n'))

**Done**